In [1]:
import geopandas as gpd
import folium
from pathlib import Path
import rasterio
import leafmap.leafmap as leafmap
import os
os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = 'proxy/{port}'

# Read parcels data

In [2]:
raw_data = Path("../data/raw_data/")

In [3]:
parcels_df = gpd.read_file(raw_data / "nl_parcels_subset_small.gpkg")
display("CRS:", parcels_df.crs)
parcels_df.head(3)

'CRS:'

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

,objectid,area,crop_type,geometry
0,1526170,51374.322841,winter_common_soft_wheat,"MULTIPOLYGON (((3.79995 51.47216, 3.80034 51.4..."
1,1526353,96633.566485,chicory_chicories,"MULTIPOLYGON (((3.76622 51.49420, 3.76622 51.4..."
2,1526354,3829.956679,sugar_beet,"MULTIPOLYGON (((3.78401 51.49023, 3.78425 51.4..."


# Read Soil Organic Carbon Map

In [4]:
print("""Overview: Soil log organic carbon content in weight fraction [g/kg]

Traceability (lineage): Predictions of soil properties were generated by using legacy soil samples and profiles for Europe, primarily from LUCAS soil, GEMAS and national soil profile databases, most importantly BZE LW German national soil profile DB. Points were overlay using spacetime reference (latitutude, longitude, year of sampling) versus some 220 time-series and static layers representing soil forming factors: climate, relief, surface reflectance, vegetation indices, hydrological indices. Ensemble Machine Learning models were fitted per soil property for the whole spacetime cube (2000-2020) and then used to predict values at 4 standard depths. For each pixel we also provide prediction error.

Scientific methodology: Predictions are based on Ensemble Machine Learning using 3D predictive soil mapping framework explained in detail in: https://opengeohub.github.io/spatial-prediction-eml/spatial-interpolation-in-3d-using-ensemble-ml.html. 3D predictive soil mapping is also explained in detail in https://soilmapper.org.

Usability: The dataset is suited for farm-scale and regional applications for monitoring soil dynamics for Europe for 2000-2020. Targeted intended uses of the data include: Land restoration and regenerative agriculture projects; An open platform for soil organic carbon monitoring; Time-series analysis of trends in soil properties and detection of positive and negative drivers of change; Uncertainty guided sampling to help improve predictions at local / regional levels;

Uncertainty quantification: Predictions errors are provided per pixel and can be used to estimate prediction intervals and mitigate dicision risks.

Data validation approaches: Prediction accuracy has been validated using 5-fold cross validation and is provided per soil property.

Completeness: The dataset covers the entire Geo-harmonizer region as defined by the landmask raster dataset. However, some small islands might be missing if there are no data in the original ERA5 Land dataset.

Consistency: Predictions are based on single pan-European models, hence it is assumed that all predictions are consistent / unbiased and no additional corrections are needed.

Positional accuracy: 30 m spatial resolution

Temporal accuracy: The maps cover the period 2000 - 2020, each map covers a certain number of years according to the following scheme: (1) 2000--2002, (2) 2002--2006, (3) 2006--2010, (4) 2010--2014, (5) 2014--2018 and (6) 2018--2020

Thematic accuracy: Raster values represent various soil properties soil carbon (g/kg), soil pH, bulk density, sand content and clay content."""
       )

Overview: Soil log organic carbon content in weight fraction [g/kg]

Traceability (lineage): Predictions of soil properties were generated by using legacy soil samples and profiles for Europe, primarily from LUCAS soil, GEMAS and national soil profile databases, most importantly BZE LW German national soil profile DB. Points were overlay using spacetime reference (latitutude, longitude, year of sampling) versus some 220 time-series and static layers representing soil forming factors: climate, relief, surface reflectance, vegetation indices, hydrological indices. Ensemble Machine Learning models were fitted per soil property for the whole spacetime cube (2000-2020) and then used to predict values at 4 standard depths. For each pixel we also provide prediction error.

Scientific methodology: Predictions are based on Ensemble Machine Learning using 3D predictive soil mapping framework explained in detail in: https://opengeohub.github.io/spatial-prediction-eml/spatial-interpolation-in-3d-u

In [5]:
carbon_tif_path = raw_data / "soil_log_organic_carbon_content_2020.tif"
with rasterio.open(carbon_tif_path) as f:
    display("Meta:", f.meta)
    raster_stats = f.statistics(bidx=1)
    display("Stats:", raster_stats)


'Meta:'

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': 255.0,
 'width': 1759,
 'height': 1194,
 'count': 1,
 'crs': CRS.from_epsg(3035),
 'transform': Affine(30.0, 0.0, 3864390.0,
        0.0, -30.0, 3193210.0)}

'Stats:'

Statistics(min=22.0, max=41.0, mean=30.491724791214505, std=2.5945111261935736)

## Visualize the data

In [6]:
m = leafmap.Map(
    basemap="CartoDB.DarkMatter",
    zoom=2,
    draw_control=False,
    measure_control=False,
    fullscreen_control=False,
    attribution_control=True,
)

m.add_gdf(parcels_df)
m.add_raster(
    str(carbon_tif_path), layer_name='Soil Organic Carbon', vmin=raster_stats.min, vmax=raster_stats.max, palette="viridis")
m

Map(center=[20, 0], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Zoom…